# Advacned CNN
高级CNN网络，包括ResNet、DenseNet，很好支持更加深层的网络，避免深层网络出现的过拟合等问题。
- **ResNet** 残差神经网络，主体是残差块，用于拟合输出和输入的残差。如果残差单元的所有参数均为0，意味着残差块为0，输出 = 输入。<br>
- **DenseNet**  稠密连接网络，和ResNet类似，只不过将稠密块的输出和输入直接在通道维度拼接。<br>
